# TinyLlama RLHF Training with PPO

이 노트북은 TinyLlama 모델을 PPO(Proximal Policy Optimization)로 파인튜닝합니다.

**Repository**: https://github.com/kaz264/tinyllama-rlhf-training

## 주요 특징
- TinyLlama-1.1B-Chat 모델 사용
- PPO 알고리즘으로 강화학습
- LoRA를 통한 효율적인 파인튜닝
- 긍정적인 감정 표현 생성 학습

## 1. 환경 설정 및 라이브러리 설치

필요한 라이브러리를 설치합니다. (약 2-3분 소요)

In [ ]:
# GPU 확인
!nvidia-smi

print("\n" + "="*50)
print("라이브러리 설치 중... (2-3분 소요)")
print("="*50 + "\n")

In [ ]:
!pip install -q torch>=2.0.0
!pip install -q transformers==4.40.0
!pip install -q trl==0.8.6
!pip install -q peft==0.10.0
!pip install -q accelerate==0.29.3
!pip install -q bitsandbytes>=0.41.0

print("✅ 모든 라이브러리 설치 완료!")

## 2. GitHub 저장소 클론

학습 코드를 다운로드합니다.

In [ ]:
# 기존 디렉토리가 있으면 삭제
!rm -rf tinyllama-rlhf-training

# 저장소 클론
!git clone https://github.com/kaz264/tinyllama-rlhf-training.git

# 작업 디렉토리 이동
%cd tinyllama-rlhf-training

print("\n✅ 저장소 클론 완료!")
!ls -la

## 3. 파일 확인

다운로드된 파일들을 확인합니다.

In [ ]:
print("📁 다운로드된 파일:")
!ls -lh

print("\n📄 train_rlhf.py 파일 확인:")
!head -20 train_rlhf.py

## 4. 학습 설정 확인 (선택 사항)

학습 설정을 확인하고 필요시 수정할 수 있습니다.

### 주요 설정값:
- **TRAINING_EPOCHS**: 100 (학습 반복 횟수)
- **TARGET_BATCH_SIZE**: 10 (배치 크기)
- **learning_rate**: 2e-5 (학습률)
- **TARGET_WORDS**: happy, glad, good, great, smile, joy, love

Colab 무료 버전에서는 시간 제한이 있으므로, 에포크를 줄이는 것을 권장합니다.

In [ ]:
# 학습 에포크를 줄이려면 아래 주석을 해제하고 실행하세요
# !sed -i 's/TRAINING_EPOCHS = 100/TRAINING_EPOCHS = 30/' train_rlhf.py

print("현재 학습 설정:")
!grep -E "TRAINING_EPOCHS|TARGET_BATCH_SIZE|learning_rate|TARGET_WORDS" train_rlhf.py

## 5. GPU 런타임 확인

**중요**: 이 노트북은 GPU 런타임이 필요합니다.

GPU가 활성화되어 있지 않다면:
1. 메뉴: **런타임** → **런타임 유형 변경**
2. **하드웨어 가속기**를 **GPU** (T4 또는 사용 가능한 GPU)로 선택
3. **저장** 클릭

In [ ]:
import torch

if torch.cuda.is_available():
    print("✅ GPU 사용 가능!")
    print(f"   디바이스: {torch.cuda.get_device_name(0)}")
    print(f"   GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  GPU를 사용할 수 없습니다!")
    print("   런타임 유형을 GPU로 변경해주세요.")

## 6. 학습 시작

이제 TinyLlama 모델을 학습시킵니다.

### 예상 소요 시간:
- GPU (T4): 약 30-60분 (100 에포크 기준)
- CPU: **권장하지 않음** (매우 느림)

### 학습 중 표시되는 정보:
- 모델 로딩 진행 상황
- 각 에포크 및 스텝 완료 알림
- 긍정적인 단어 생성 발견 시 🎉 알림
- 10 에포크마다 진행 상황 및 성공률

**참고**: 첫 실행 시 모델 다운로드로 인해 시작이 느릴 수 있습니다.

In [ ]:
# 학습 실행
!python train_rlhf.py

## 7. 학습 결과 확인

학습이 완료되면 생성된 모델과 결과를 확인합니다.

In [ ]:
import os

if os.path.exists("./trained_model"):
    print("✅ 학습된 모델이 저장되었습니다!\n")
    print("저장된 파일:")
    !ls -lh ./trained_model/
else:
    print("⚠️  모델 파일을 찾을 수 없습니다. 학습이 완료되었는지 확인하세요.")

## 8. 학습된 모델 테스트

학습된 모델로 텍스트를 생성해봅니다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

print("학습된 모델 로딩 중...\n")

# 모델 로드
tokenizer = AutoTokenizer.from_pretrained("./trained_model")
model = AutoModelForCausalLM.from_pretrained("./trained_model", device_map="auto")

print("✅ 모델 로드 완료!\n")
print("="*50)
print("학습된 모델 테스트 결과")
print("="*50 + "\n")

# 테스트 쿼리
test_queries = [
    "I feel so",
    "This makes me",
    "Today I am",
    "I am really",
    "The weather is"
]

for query in test_queries:
    inputs = tokenizer(query, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=20,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"입력: '{query}'")
    print(f"생성: '{generated_text}'\n")

print("완료! 🎉")

## 9. 학습된 모델 다운로드 (선택 사항)

학습된 모델을 로컬로 다운로드하려면 아래 셀을 실행하세요.

In [ ]:
# 모델을 압축하여 다운로드
!zip -r trained_model.zip ./trained_model/

print("\n✅ 모델이 압축되었습니다!")
print("왼쪽 파일 탐색기에서 'trained_model.zip'을 찾아 다운로드하세요.")

# Colab에서 직접 다운로드
from google.colab import files
files.download('trained_model.zip')

## 다음 단계

### 학습 개선 방법:
1. **에포크 증가**: `TRAINING_EPOCHS`를 150-200으로 늘리기
2. **학습률 조정**: `learning_rate`를 3e-5로 높이기
3. **보상 함수 수정**: 더 다양한 긍정 단어 추가
4. **쿼리 확장**: 더 많은 입력 문장 추가

### 추가 실험:
- 다른 감정 (슬픔, 분노 등)으로 학습
- 더 큰 모델 사용 (예: Llama-2-7B)
- DPO (Direct Preference Optimization) 시도

---

**문제가 발생하면**: [GitHub Issues](https://github.com/kaz264/tinyllama-rlhf-training/issues)에 문의하세요!

Happy Training! 🚀